In [1]:
using JuMP, Gurobi, IPG

In [2]:
n = m = 2

X = [Strategies(1, 0, 0), Strategies(1, 0, 0)]
Π = [QuadraticPayoff(0, [2, 1]), QuadraticPayoff(0, [1, 2])]
;

In [3]:
x = [[0],[0]]
payoff(Π, x, 1)

0.0

# Step 1: Initialization

Compute one pure strategy for each player. Build sampled normal-form game.

In [4]:
function find_feasible_pure_strategy(Xp::Strategies, Πp::QuadraticPayoff)
    Ap, bp, Bp = Xp.Ap, Xp.bp, Xp.Bp

    # Create a new model to solve the feasibility problem
    m = Model(Gurobi.Optimizer)
    @variable(m, xp[1:np(Xp)])
    for i in 1:Bp  # TODO: there's probably a cleaner way to do this
        set_binary(xp[i])
    end
    @constraint(m, Ap * xp <= bp)
    @objective(m, Min, 0)  # Feasibility!

    # TODO: I'll probably want to silence this sometime in the future
    optimize!(m)

    return value.(xp)
end

function find_feasible_pure_profile(X::Vector{Strategies}, Π::Vector{QuadraticPayoff})
    return [find_feasible_pure_strategy(Xp, Πp) for (Xp, Πp) in zip(X, Π)]
end

x = find_feasible_pure_profile(X, Π)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562957
Academic license 2562957 - for non-commercial use only - registered to br___@umontreal.ca
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (armlinux64 - "Ubuntu 22.04.5 LTS")

CPU model: ARM64
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license 2562957 - for non-commercial use only - registered to br___@umontreal.ca
Optimize a model with 1 rows, 1 columns and 1 nonzeros
Model fingerprint: 0xe4a4e4c9
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  

2-element Vector{Vector{Float64}}:
 [0.0]
 [0.0]

In [5]:
# sample of strategies
S_X = [Vector{Vector{Float64}}() for p in 1:m]
# S_X[p,j,i] is the i-th dimension of the j-th strategy of player p

for p in 1:m
    # add to sample the new strategy
    push!(S_X[p], x[p])
end

In [6]:
for p in 1:m
    # add to sample the new strategy
    push!(S_X[p], [1.0])
end

In [10]:
function get_polymatrix(S_X::Vector{<:Vector{<:Vector{<:Real}}}, Π::Vector{QuadraticPayoff})
    m = length(S_X)

    dims = [length(S_X[p]) for p in 1:m]

    polymatrix = Dict{Tuple{Integer, Integer}, Matrix{Float64}}()

    for p in 1:m
        for k in 1:m
            polymatrix[p,k] = zeros(length(S_X[p]), length(S_X[k]))

            if k == p
                for i in length(S_X[p])
                    polymatrix[p,p][i,i] = IPG.bilateral_payoff(Π[p], p, S_X[p][i], p, S_X[p][i])
                end
            else
                for i in 1:length(S_X[p])
                    for j in 1:length(S_X[k])
                        polymatrix[p,k][i,j] = IPG.bilateral_payoff(Π[p], p, S_X[p][i], k, S_X[k][j])
                    end
                end
            end
        end
    end

    return polymatrix
end

polymatrix = get_polymatrix(S_X, Π)

Dict{Tuple{Integer, Integer}, Matrix{Float64}} with 4 entries:
  (1, 2) => [0.0 0.0; 0.0 1.0]
  (1, 1) => [0.0 0.0; 0.0 -1.0]
  (2, 2) => [0.0 0.0; 0.0 -1.0]
  (2, 1) => [0.0 0.0; 0.0 1.0]

In [11]:
using NormalGames

G = NormalGames.NormalGame(m, length.(S_X), polymatrix)

NormalGames.NormalGame(2, [2, 2], Dict((1, 2) => [0.0 0.0; 0.0 1.0], (1, 1) => [0.0 0.0; 0.0 -1.0], (2, 2) => [0.0 0.0; 0.0 -1.0], (2, 1) => [0.0 0.0; 0.0 1.0]))

# Step 2: Solve sampled game

Compute a solution to the normal-form game with the sampled strategies.

In [12]:
# TODO